# 1. Library Import

In [ ]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

## Optuna tunning for XGB
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import optuna

## Optuna visulization
import plotly.express as px
import plotly.graph_objects as go
import plotly


# 2. Data Loading

In [ ]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = '../data/train_latlon.csv'
test_path  = '../data/test_latlon.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train1 = train.copy()
test1 = test.copy()

# 3. Preprocessing Data

### 3-1. Train

In [ ]:
train1.loc[(train1['address'].isna()) & (train1['name'] == '서초포레스타2단지'), 'address'] = 384
# train1[(train1['city'] == '서울특별시 서초구 내곡동')&(train['name']=='서초포레스타2단지')]

train1.loc[(train1['address'].isna()) & (train1['name'] == '힐스테이트 서초 젠트리스'), 'address'] = 557
# train1[(train1['city'] == '서울특별시 서초구 신원동')&(train['name']=='힐스테이트 서초 젠트리스')]

In [ ]:
train1.loc[(train1['first_num'].isna()) & (train1['name'] == '힐스테이트 서초 젠트리스'), 'first_num'] = 557

train1.loc[(train1['second_num'].isna()) & (train1['name'] == '힐스테이트 서초 젠트리스'), 'second_num'] = 0

In [ ]:
# 'city' 열을 공백을 기준으로 분할하여 'gu'와 'dong' 열 생성
train1[['gu', 'dong']] = train1['city'].str.split(expand=True)[[1, 2]]

In [ ]:
# train1 데이터프레임에서 'name' 열이 결측치인 행 선택
missing_name_rows = train1[train1['name'].isna()]

# 선택된 행들에 대해 '구'+'first_num'+'_'+'second_num'로 결측치 대입
missing_name_rows['name'] = missing_name_rows['dong'] + missing_name_rows['first_num'].astype(str) + '_' + missing_name_rows['second_num'].astype(str)

# 기존 train1에 변경된 행들을 업데이트
train1.update(missing_name_rows)

### 3-2. Test 

# 'city' 열을 공백을 기준으로 분할하여 'gu'와 'dong' 열 생성
test1[['gu', 'dong']] = test1['city'].str.split(expand=True)[[1, 2]]

In [ ]:
# train1 데이터프레임에서 'name' 열이 결측치인 행 선택
missing_name_rows = test1[test1['name'].isna()]

# 선택된 행들에 대해 '구'+'first_num'+'_'+'second_num'로 결측치 대입
missing_name_rows['name'] = missing_name_rows['dong'] + missing_name_rows['first_num'].astype(str) + '_' + missing_name_rows['second_num'].astype(str)

# 기존 train1에 변경된 행들을 업데이트
test1.update(missing_name_rows)

# 4. BaseLine Modeling

### 4-1. Data Preprocessing

# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
train1['is_test'] = 0
test1['is_test'] = 1
concat = pd.concat([train1, test1])     # 하나의 데이터로 만들어줍니다.

In [ ]:
concat['is_test'].value_counts()      # train과 test data가 하나로 합쳐진 것을 확인할 수 있습니다.

### 4-2. 결측치 탐색 및 보간

#### 4-2-1. 결측치 탐색

In [ ]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
concat['registration_day'] = concat['registration_day'].replace(' ', np.nan)
concat['transaction_type'] = concat['transaction_type'].replace('-', np.nan)
concat['estate_agent'] = concat['estate_agent'].replace('-', np.nan)

In [ ]:
# EDA에 앞서 결측치를 확인해보겠습니다.
concat.isnull().sum()

In [ ]:
# 변수별 결측치의 비율을 plot으로 그려보면 아래와 같습니다.
fig = plt.figure(figsize=(13, 2))
missing = concat.isnull().sum() / concat.shape[0]
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar(color='orange')
plt.title('변수별 결측치 비율')
plt.show()

In [ ]:
# Null값이 100만개 이상인 칼럼은 삭제해보도록 하겠습니다.
print('* 결측치가 100만개 이하인 변수들 :', list(concat.columns[concat.isnull().sum() <= 1000000]))     # 남겨질 변수들은 아래와 같습니다.
print('* 결측치가 100만개 이상인 변수들 :', list(concat.columns[concat.isnull().sum() >= 1000000]))

In [ ]:
# 위에서 결측치가 100만개 이하인 변수들만 골라 새로운 concat_select 객체로 저장해줍니다.
selected = list(concat.columns[concat.isnull().sum() <= 1000000])
concat_select = concat[selected]

In [ ]:
concat_select.isnull().sum()     # 결측치가 100만개 초과인 칼럼이 제거된 모습은 아래와 같습니다.
# target변수는 test dataset 개수만큼(9272) 결측치가 존재함을 확인할 수 있습니다.

#### 4-2-2. 선형 보간 

In [ ]:
# 연속형 변수는 선형보간을 해주고, 범주형변수는 알수없기에 “unknown”이라고 임의로 보간해 주겠습니다.
concat_select.info()

In [ ]:
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸어주고 아래 작업을 진행하겠습니다.
concat_select['first_num'] = concat_select['first_num'].astype('str')
concat_select['second_num'] = concat_select['second_num'].astype('str')

In [ ]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in concat_select.columns:
    if pd.api.types.is_numeric_dtype(concat_select[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

In [ ]:
# 범주형 변수에 대한 보간
concat_select[categorical_columns] = concat_select[categorical_columns].fillna('NULL')

# 연속형 변수에 대한 보간 (선형 보간)
concat_select[continuous_columns] = concat_select[continuous_columns].interpolate(method='linear', axis=0)

### 4-3. 이상치 처리

In [ ]:
# 대표적인 연속형 변수인 “전용 면적” 변수 관련한 분포를 먼저 살펴보도록 하겠습니다.
fig = plt.figure(figsize=(7, 3))
sns.boxplot(data = concat_select, x = 'area', color='lightgreen')
plt.title('area 분포')
plt.xlabel('Area')
plt.show()

In [ ]:
# 이상치 제거 방법에는 IQR을 이용하겠습니다.
def remove_outliers_iqr(dt, column_name):
    train1 = dt.query('is_test == 0')       # train data 내에 있는 이상치만 제거하도록 하겠습니다.
    test1 = dt.query('is_test == 1')

    Q1 = train1[column_name].quantile(0.25)
    Q3 = train1[column_name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    train1 = train1[(train1[column_name] >= lower_bound) & (train1[column_name] <= upper_bound)]

    result = pd.concat([train1, test1])   # test data와 다시 합쳐주겠습니다.
    return result

In [ ]:
# 위 방법으로 전용 면적에 대한 이상치를 제거해보겠습니다.
concat_select = remove_outliers_iqr(concat_select, 'area')

In [ ]:
# 이상치 제거 후의 shape은 아래와 같습니다. 약 10만개의 데이터가 제거된 모습을 확인할 수 있습니다.
print(concat_select.shape)

In [ ]:
concat_select['is_test'].value_counts()     # 또한, train data만 제거되었습니다.

### 4-4. Feature Engineering

In [ ]:
concat_select['year_contract'] = concat_select['contract_year'].astype('str').map(lambda x : x[:4])
concat_select['month_contract'] = concat_select['contract_year'].astype('str').map(lambda x : x[4:])
del concat_select['contract_year']

In [ ]:
all = list(concat_select['gu'].unique())
gangnam = ['강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', '강동구']
gangbuk = [x for x in all if x not in gangnam]

assert len(all) == len(gangnam) + len(gangbuk)       # 알맞게 분리되었는지 체크합니다.

In [ ]:
# 건축년도 분포는 아래와 같습니다. 특히 2005년이 Q3에 해당합니다.
# 2009년 이후에 지어진 건물은 10%정도 되는 것을 확인할 수 있습니다.
concat_select['construction_year'].describe(percentiles = [0.1, 0.25, 0.5, 0.75, 0.8, 0.9])

In [ ]:
# 따라서 2009년 이후에 지어졌으면 비교적 신축이라고 판단하고, 신축 여부 변수를 제작해보도록 하겠습니다.
concat_select['new_construction'] = concat_select['construction_year'].apply(lambda x: 1 if x >= 2009 else 0)

# 5. Model Training

# 이제 다시 train과 test dataset을 분할해줍니다. 위에서 제작해 놓았던 is_test 칼럼을 이용합니다.
dt_train = concat_select.query('is_test==0')
dt_test = concat_select.query('is_test==1')

# 이제 is_test 칼럼은 drop해줍니다.
dt_train.drop(['is_test'], axis = 1, inplace=True)
dt_test.drop(['is_test'], axis = 1, inplace=True)
print(dt_train.shape, dt_test.shape)

In [ ]:
# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
dt_test['target'] = 0

### 5-1. 범주형 변수 Encoding

# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in dt_train.columns:
    if pd.api.types.is_numeric_dtype(dt_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

In [ ]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( dt_train[col].astype(str) )
    dt_train[col] = lbl.transform(dt_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(dt_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    dt_test[col] = lbl.transform(dt_test[col].astype(str))

In [ ]:
assert dt_train.shape[1] == dt_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [ ]:
# Target과 독립변수들을 분리해줍니다.
# y_train = dt_train['target']
y = np.log1p(dt_train['target'])
X = dt_train.drop(['target'], axis=1)

# Hold out split을 사용해 학습 데이터와 검증 데이터를 8:2 비율로 나누겠습니다.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1996)

### 5-2. 학습 전 Data Heatmap Check

In [ ]:
plt.figure(figsize=(30, 20))

mask = np.triu(np.ones_like(concat_select.corr(), dtype=bool))
sns.heatmap(concat_select.corr(), annot=True, mask=mask)
plt.show()

### 5-3. Model Train

## Optuna tunning for XGB
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import optuna

def objective(trial):
    params = {
    'n_estimators': trial.suggest_int('n_estimators', 50,1000),
    'max_depth': trial.suggest_int('max_depth', 3, 100),
    'min_child_weight': trial.suggest_float('min_child_weight', 2,100),
    'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.2, log = True),
    'subsample': trial.suggest_float('subsample', 0.2, 1),
    'gamma': trial.suggest_float('gamma', 1e-3, 1.0),
    'colsample_bytree': trial.suggest_float('colsample_bytree',0.2,1),
    'colsample_bylevel': trial.suggest_float('colsample_bylevel',0.2,1),
    'colsample_bynode': trial.suggest_float('colsample_bynode', 0.2, 1)
    }
    
    xgbmodel_optuna = XGBRegressor(**params, random_state = 1996, tree_method = 'gpu_hist')
#     xgbmodel_optuna = XGBClassifier(**params, random_state = 1996)

    # K-Fold 
    cv = abs(cross_val_score(xgbmodel_optuna, X, y, cv = 5,scoring='neg_mean_squared_error').mean())
    return cv

In [ ]:
%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
# Best trial: score 0.03265444509960992, 
# params {'n_estimators': 488, 'max_depth': 50, 'min_child_weight': 48.945864172343384, 'learning_rate': 0.030697380883251864, 'subsample': 0.6121133384530306, 'gamma': 0.5246758849926558, 'colsample_bytree': 0.41215901060635096, 'colsample_bylevel': 0.6721620802187772, 'colsample_bynode': 0.3964629902502232}

# Best trial: score 0.0177888242405573, 
# params {'n_estimators': 80, 'max_depth': 10, 'min_child_weight': 25.99248047074334, 'learning_rate': 0.1842939934421537, 'subsample': 0.5542525905300756, 'gamma': 0.056225796442002485, 'colsample_bytree': 0.8759593051211264, 'colsample_bylevel': 0.884009475162997, 'colsample_bynode': 0.8146897367191535}

# Best trial: score 0.020571836382108914, 
# params {'n_estimators': 888, 'max_depth': 16, 'min_child_weight': 75.38142286670956, 'learning_rate': 0.01714887049975046, 'subsample': 0.3029506390662331, 'gamma': 0.04078329280278583, 'colsample_bytree': 0.7497748603288371, 'colsample_bylevel': 0.685499538370593, 'colsample_bynode': 0.9776152134092071}

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value,study.best_trial.params))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly

# Hyperparameter Importances를 통해서 parameter를 고정시켜라.
# 그리고 나머지 것들을 진행시켜라.
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
xgb_params = study.best_trial.params

In [ ]:
# model finalization
# optuna는 hyperparameter를 조정.

print("Val ACC")
best_params = study.best_params
best_model = XGBRegressor(**xgb_params, 
                        random_state=1996, 
                        tree_method = 'gpu_hist',
                       )

best_model.fit(X_train, y_train)
pred = best_model.predict(X_val)
# 회귀 관련 metric을 통해 train/valid의 모델 적합 결과를 관찰합니다.
print(f'RMSE test: {np.sqrt(metrics.mean_squared_error(y_val, pred))}')

In [ ]:
# 학습된 모델을 저장합니다. Pickle 라이브러리를 이용하겠습니다.
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
dt_test.head(2)      # test dataset에 대한 inference를 진행해보겠습니다.

In [ ]:
# 저장된 모델을 불러옵니다.
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
%%time
X_test = dt_test.drop(['target'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = model.predict(X_test)

In [ ]:
submission = pd.read_csv('../data/sample_submission.csv')

In [ ]:
submission['target'] = np.expm1(real_test_pred)

In [ ]:
# 올림
submission['target'] = np.ceil(submission['target']).astype(int)

# 반올림
# submission['target'] = round(submission['target']).astype(int)

# 버림
# submission['target'] = np.floor(submission['target']).astype(int)


In [ ]:
submission.to_csv('../data/xgb_best_model_log1p_올림.csv', index=False)